# **Encoder-decoder model for sequence to sequence translation from German to English**

## Import modules and packages
Import all the necessary modules and packages necessary to build, train, and evaluate the model. Also, set random seeds so that the program can be run for multiple seeds to check how variant the model's performance is.

In [17]:
#Add the Google Drive location of all files to system path
import os,sys
from google.colab import drive
drive.mount('/content/drive/')
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE='Colab Notebooks/Sequence to sequence'
GOOGLE_DRIVE_PATH=os.path.join('drive','My Drive',GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))
sys.path.append(GOOGLE_DRIVE_PATH)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
['2 - Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation.ipynb', '3 - Neural Machine Translation by Jointly Learning to Align and Translate.ipynb', 'requirements.txt', 'README.md', 'assets', 'legacy', '1 - Sequence to Sequence Learning with Neural Networks.ipynb']


In [18]:
!pip install datasets evaluate --upgrade
!python -m spacy download de_core_news_sm
!pip install -U torchtext==0.15.2
!pip install -r requirements.txt

#Download the tokeniers for Englisha and German
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

import collections,random,time
import numpy as np
import datasets
#import evaluate
import spacy
import tqdm
import torchtext
import torchtext.vocab as ttv
import torch
import torch.nn as nn
import torch.optim as optim
import torch.functional as F
from torch.utils.data import Dataset

#Set the environment time
os.environ['TZ/']='US/Central'
time.tzset()

#Set the seed
sd=random.randint(0,50)
sd=1234
random.seed(sd)
np.random.seed(sd)
torch.manual_seed(sd)
torch.cuda.manual_seed(sd)
torch.backends.cudnn.deterministic=True

#Check GPU availability
if torch.cuda.is_available():
  print('\n\nThe GPU available is a {}.'.format(torch.cuda.get_device_name(0)))
  print('\nProgram execution started on: {}'.format(time.ctime()))
else:print('To ensure GPU availability, try Edit -> Notebook Settings.')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 61.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
  Using cached torch-2.0.1-cp310-cp310-manylinux1_x86_64.whl.metadata (24 kB)
  Using cached triton-2.0.0-1-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.0 kB)
Using cached torch-2.0.1-cp310-cp310-manylinux1_x86_64.whl (619.9 MB)
Using cached triton-2.0.0-1-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (63.3 MB)
  Attempting uninstall: triton
    Found existing installation: triton 3.0.0
    Uninstalling triton-3.0.0:
      Successfully uninstalled triton-3.0.0
  Attempting uninstall: torch
    Found existing installation: torch 2.4.1
   

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 68.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
  Using cached https://github.com/explosion/spacy-models/releases/download/de_core_news_sm-3.7.0/de_core_news_sm-3.7.0-py3-none-any.whl (14.6 MB)
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


The GPU available is a Tesla T4.

Program execution started on: Fri Oct 11 07:10:44 2024


## The dataset
The dataset used is a subset of [Multi30k dataset](https://github.com/multi30k/dataset), which is from HuggingFace and hosted at [the page](https://huggingface.co/datasets/bentrevett/multi30k). The dataset contains about 30K parallel English and German sentences.


### Define the dataset
Download the 'train' and 'test' splits of the dataset, and inspect its characteristics.

In [19]:
#Download the 'train', the 'validation', and the 'test' splits of the dataset
trSet,valSet,tsSet=datasets.load_dataset('bentrevett/multi30k',split=['train','validation','test'])
print('The respective sizes of the train, the validation, and the test datasets are {}, {}, and {} respectively.'.format(trSet.num_rows,valSet.num_rows,tsSet.num_rows))
print('A sample picked from the train dataset would look as shown below:\n\n{}'.format(trSet[0]))

The respective sizes of the train, the validation, and the test datasets are 29000, 1014, and 1000 respectively.
A sample picked from the train dataset would look as shown below:

{'en': 'Two young, White males are outside near many bushes.', 'de': 'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.'}


### Tokenization
Convert the datasets from collections of strings to collections of tokens so that it is possible for any machine learning model to process them after the tokens are transformed into numbers.

In [20]:
#The function that tokenizes the datasets
def tokenize(dataset,enTokenizer,deTokenizer,maxLn,lower,sos,eos):
  enTokens=[token.text for token in enTokenizer.tokenizer(dataset['en'])][:maxLn]
  deTokens=[token.text for token in deTokenizer.tokenizer(dataset['de'])][:maxLn]
  if lower:                              #convert all uppercase tokens to lower if 'lower' is set
    enTokens=[token.lower() for token in enTokens]
    deTokens=[token.lower() for token in deTokens]
  enTokens=[sos]+enTokens+[eos]
  deTokens=[sos]+deTokens+[eos]
  return {'enTokens':enTokens,'deTokens':deTokens}

#Define the tokenizer and perform tokenization of the datasets
maxLn=1000                               #maximum length of each token
lower=True                               #whether to make all tokens lowercase or not
sosToken,eosToken='<sos>','<eos>'        #start and end of string tokens
enTokenizer=spacy.load('en_core_web_sm') #use 'tokenizer' in enTokenizer to tokenize
deTokenizer=spacy.load('de_core_news_sm')#use 'tokenizer' in deTokenizer to tokenize
print('Tokenizing the train dataset...')
trSet=trSet.map(tokenize,fn_kwargs={'enTokenizer':enTokenizer,'deTokenizer':deTokenizer,'maxLn':maxLn,'lower':lower,'sos':sosToken,'eos':eosToken})
print('Tokenizing the validation dataset...')
valSet=valSet.map(tokenize,fn_kwargs={'enTokenizer':enTokenizer,'deTokenizer':deTokenizer,'maxLn':maxLn,'lower':lower,'sos':sosToken,'eos':eosToken})
print('Tokenizing the test dataset...')
tsSet=tsSet.map(tokenize,fn_kwargs={'enTokenizer':enTokenizer,'deTokenizer':deTokenizer,'maxLn':maxLn,'lower':lower,'sos':sosToken,'eos':eosToken})
print('\nThe contents of the tokenized datasets are as follows:\n{}'.format(trSet))
print('\nA sample from the tokenized train dataset would look as shown below:\n1. English:\n{}'.format(trSet[0]['en']))
print('2. English tokens:\n{}'.format(trSet[0]['enTokens']))
print('3. German:\n{}'.format(trSet[0]['de']))
print('4. German tokens:\n{}'.format(trSet[0]['deTokens']))

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


Tokenizing the train dataset...
Tokenizing the validation dataset...


Map:   0%|          | 0/1014 [00:00<?, ? examples/s]

Tokenizing the test dataset...


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]


The contents of the tokenized datasets are as follows:
Dataset({
    features: ['en', 'de', 'enTokens', 'deTokens'],
    num_rows: 29000
})

A sample from the tokenized train dataset would look as shown below:
1. English:
Two young, White males are outside near many bushes.
2. English tokens:
['<sos>', 'two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.', '<eos>']
3. German:
Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.
4. German tokens:
['<sos>', 'zwei', 'junge', 'weiße', 'männer', 'sind', 'im', 'freien', 'in', 'der', 'nähe', 'vieler', 'büsche', '.', '<eos>']


### Vocabulary
Convert the tokens into one-hot representations by defining a minimum threshold frequency of occurence for each token and the special tokens consisting of the unknown, the padding tokens, the SOS, and the EOS tokens.

In [21]:
#Define the vocabularies for English and German
threshold=2                                   #minimum frequency of occurence for a token to be assigned a one-hot representation
spclTokens=['<unk>','<pad>',sosToken,eosToken]#to represent tokens with frequency<threshold and for padding to make all sentences have the same lengths
enVocabulary=ttv.build_vocab_from_iterator(trSet['enTokens'],min_freq=threshold,specials=spclTokens)                       #English vocabulary
deVocabulary=ttv.build_vocab_from_iterator(trSet['deTokens'],min_freq=threshold,specials=spclTokens)                       #German vocabulary
print('The number of tokens in the vocabularies are as follows:\n1. English: {}\n2. German: {}'.format(len(enVocabulary),len(deVocabulary)))

#The special tokens in the vocabularies
enIdxUnk,enIdxPad,enIdxSos,enIdxEos=enVocabulary['<unk>'],enVocabulary['<pad>'],enVocabulary['<sos>'],enVocabulary['<eos>']#indices of special tokens in English vocabulary
deIdxUnk,deIdxPad,deIdxSos,deIdxEos=deVocabulary['<unk>'],deVocabulary['<pad>'],deVocabulary['<sos>'],deVocabulary['<eos>']#indices of special tokens in German vocabulary
assert enIdxUnk==deIdxUnk                     #in both vocabularies, the special tokens will be at indices 0,1,2, and 3 in the order they appear in 'spclTokens'. So, assert that
assert enIdxPad==deIdxPad
assert enIdxSos==deIdxSos
assert enIdxEos==deIdxEos
enVocabulary.set_default_index(enIdxUnk)      #set the default token for the English vocabulary
deVocabulary.set_default_index(deIdxUnk)      #set the default token for the German vocabulary

The number of tokens in the vocabularies are as follows:
1. English: 5893
2. German: 7853


### Numerize the vocabulary
Convert the tokens into a numerical format so that they can be processed by neural networks.

In [22]:
#The function that numerizes the tokenized dataset
def numerize(dataset,enVocab,deVocab):
  enIndices=enVocab.lookup_indices(dataset['enTokens'])
  deIndices=deVocab.lookup_indices(dataset['deTokens'])
  return {'enIndices':enIndices,'deIndices':deIndices}

#Numerize the datasets
print('Numerizing the training dataset...')
trSet=trSet.map(numerize,fn_kwargs={'enVocab':enVocabulary,'deVocab':deVocabulary})
print('Numerizing the validation dataset...')
valSet=valSet.map(numerize,fn_kwargs={'enVocab':enVocabulary,'deVocab':deVocabulary})
print('Numerizing the testing dataset...')
tsSet=tsSet.map(numerize,fn_kwargs={'enVocab':enVocabulary,'deVocab':deVocabulary})
print('The contents of the numerized datasets are as follows:\n{}.'.format(trSet))
print('\nA sample from the numerized dataset would look as shown below:\n1. English:\n{}'.format(trSet[0]['en']))
print('2. English tokens:\n{}'.format(trSet[0]['enTokens']))
print('3. English indices:\n{}'.format(trSet[0]['enIndices']))
print('4. German:\n{}'.format(trSet[0]['de']))
print('5. German tokens:\n{}'.format(trSet[0]['deTokens']))
print('6. German indices:\n{}'.format(trSet[0]['deIndices']))

#Convert the datasets to tensors after retaining only 'label' and 'indices'
trSet=trSet.with_format(type='torch',columns=['enIndices','deIndices'],output_all_columns=True)
valSet=valSet.with_format(type='torch',columns=['enIndices','deIndices'],output_all_columns=True)
tsSet=tsSet.with_format(type='torch',columns=['enIndices','deIndices'],output_all_columns=True)
print('\nThe numerized datasets\' contents after their conversion to tensors are as follows:\n{}.'.format(trSet))
print('\nA sample from the converted dataset would look as shown below:\n1. English:\n{}'.format(trSet[0]['en']))
print('2. English tokens:\n{}'.format(trSet[0]['enTokens']))
print('3. English indices:\n{}'.format(trSet[0]['enIndices']))
print('4. German:\n{}'.format(trSet[0]['de']))
print('5. German tokens:\n{}'.format(trSet[0]['deTokens']))
print('6. German indices:\n{}'.format(trSet[0]['deIndices']))

Numerizing the training dataset...
Numerizing the validation dataset...


Map:   0%|          | 0/1014 [00:00<?, ? examples/s]

Numerizing the testing dataset...


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

The contents of the numerized datasets are as follows:
Dataset({
    features: ['en', 'de', 'enTokens', 'deTokens', 'enIndices', 'deIndices'],
    num_rows: 29000
}).

A sample from the numerized dataset would look as shown below:
1. English:
Two young, White males are outside near many bushes.
2. English tokens:
['<sos>', 'two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.', '<eos>']
3. English indices:
[2, 16, 24, 15, 25, 778, 17, 57, 80, 202, 1312, 5, 3]
4. German:
Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.
5. German tokens:
['<sos>', 'zwei', 'junge', 'weiße', 'männer', 'sind', 'im', 'freien', 'in', 'der', 'nähe', 'vieler', 'büsche', '.', '<eos>']
6. German indices:
[2, 18, 26, 253, 30, 84, 20, 88, 7, 15, 110, 7647, 3171, 4, 3]

The numerized datasets' contents after their conversion to tensors are as follows:
Dataset({
    features: ['en', 'de', 'enTokens', 'deTokens', 'enIndices', 'deIndices'],
    num_rows: 29000
}).

A sam

### Data loader
The data loaders are iterated upon to return a batch of data. Each batch will be a dictionary containing numericalized English and German sentences represented as tensors.

In [23]:
#The class prepares batches of data that can be iterated over by the data loader.
#The batch is prepared by collating a group of data samples in the function
#'collate'. The function also pads the shorter sequences in the batch with '<pad>'
#to make their lengths equal to that of the batch's longest sequence. This
#ensures that the neural network model's I/P dimension remains fixed.
class languageDataset(Dataset):
  def __init__(self,dataset,batchSize,enIdxPad,deIdxPad,shuffle=False) -> None:
    self.dataset,self.batchSize=dataset,batchSize
    self.enIdxPad,self.deIdxPad=enIdxPad,deIdxPad
    self.shuffle=shuffle

  def collate(self,batch):
    batchEnIndices=[sample['enIndices'] for sample in batch]
    batchDeIndices=[sample['deIndices'] for sample in batch]
    batchEnIndices=nn.utils.rnn.pad_sequence(batchEnIndices,padding_value=self.enIdxPad)
    batchDeIndices=nn.utils.rnn.pad_sequence(batchDeIndices,padding_value=self.deIdxPad)
    print('Batch sizes for English and German are {} and {} respectively.'.format(batchEnIndices.shape,batchDeIndices.shape))
    return {'enIndices':batchEnIndices,'deIndices':batchDeIndices}

  def loadData(self):
    dataLoader=torch.utils.data.DataLoader(dataset=self.dataset,batch_size=self.batchSize,collate_fn=self.collate,shuffle=self.shuffle)
    return dataLoader

def dataIterator(set,dataset,params):
  if set=='train':obj=languageDataset(dataset,params['batchSize'],params['enIdxPad'],params['deIdxPad'],shuffle=True)
  elif set=='validation':obj=languageDataset(dataset,params['batchSize'],params['enIdxPad'],params['deIdxPad'],shuffle=True)
  elif set=='test':obj=languageDataset(dataset,params['batchSize'],params['enIdxPad'],params['deIdxPad'],shuffle=True)
  return obj.loadData()

## Sequence-to-sequence LSTM model for language translation
The language translation model will use an LSTM-based encoder-decoder network arcitecture.

### The encoder
The model's encoder consists of an embedding layer, which accepts index representations of the source language, and one or more layers of LSTM units that generate context vectors to feedi the model's decoder.

In [24]:
class Encoder(nn.Module):
  #Initialize the Encoder class
  def __init__(self,ipDim,embDim,hiddenDim,L,rate,bi=False,batchFirst=False) -> None:
    super(Encoder,self).__init__()
    self.hiddenDim,self.L=hiddenDim,L
    self.embed=nn.Embedding(ipDim,embDim)#I/P dimension is the length of the source language's vocabulary
    self.lstm=nn.LSTM(embDim,hiddenDim,L,bidirectional=bi,dropout=rate,batch_first=batchFirst)
    self.dropout=nn.Dropout(rate)

  #Forward propagates a batch of indices (denoted by 'x') of size
  #[seqLength,batchSize] through the network
  def forward(self,x):
    embedding=self.dropout(self.embed(x))#size is [seqLength,batchSize,embDim]
    #Output sizes of the LSTM layer are:
    #1. 'op': ht of all last layer units i.e., h^L_t; t=0,1,2,...,T. Its size is [seqLength,batchSize,hiddenDim*nDir]; 'nDir'=2 for bidirectional and 1 otherwise
    #2. 'h': hT of all layers stacked together i.e., ht of the last units of all layers. Its size is [L*nDir,batchSize,hiddenDim]; 'nDir'=2 for bidirectional and 1 otherwise
    #3. 'c': cT of all layers stacked together i.e., ct of the last units of all layers. Its size is [L*nDir,batchSize,hiddenDim]; 'nDir'=2 for bidirectional and 1 otherwise
    op,(h,c)=self.lstm(embedding)        #h and c together form z, the context vector feed for the decoder. If initial 'h' and 'c' not given, nn.LSTM assumes them to be all zeros
    return h,c

### The decoder
The model's decoder consists of an embedding layer that accepts index representations of the true/model-predicted target language (based on teacher-forcing learning) and the same number of LSTM layers as the encoder, which predict a target-language token for the current time step with the help of a dense layer mapping.

In [25]:
class Decoder(nn.Module):
  #Initializes the Decoder class
  def __init__(self,opDim,embDim,hiddenDim,L,rate,bi=False,batchFirst=False) -> None:
    super(Decoder,self).__init__()
    self.opDim,self.hiddenDim,self.L=opDim,hiddenDim,L
    self.embed=nn.Embedding(opDim,embDim) #O/P dimension is the length of the target language's vocabulary
    self.lstm=nn.LSTM(embDim,hiddenDim,L,bidirectional=bi,dropout=rate,batch_first=batchFirst)
    self.linear=nn.Linear(hiddenDim,opDim)#O/P dimension is the length of the target language's vocabulary
    self.dropout=nn.Dropout(rate)

  #Forward propagates a batch of context vectors (denoted by 'x') through the
  #network
  def forward(self,x,h,c):
    #Input sizes of the decoder are:
    #1. 'x': True/predicted (based on teacher-forcing) word for time t-1. Its size is [batchSize], and it covers only the immediate previous time step since decoder translates one time step at a time.
    #2. 'h': hT of all layers of the encoder stacked together i.e., ht of the last units of all layers in the encoder. Its size is [L*nDir,batchSize,hiddenDim]; 'nDir'=2 for bidirectional and 1 otherwise
    #3. 'c': cT of all layers of the encoder stacked together i.e., ct of the last units of all layers in the encoder. Its size is [L*nDir,batchSize,hiddenDim]; 'nDir'=2 for bidirectional and 1 otherwise
    #'h' and 'c' form the context vector I/P for the decoder
    x=x.unsqueeze(0)                      #x's size changed to [1,batchSize] to explicitly represent the sequence length dimension for processing (seqLength=1 as x covers only the immediate previous time step).
    embedding=self.dropout(self.embed(x)) #size is [1,batchSize,embDim]
    #Output sizes of the LSTM layer are:
    #1. 'op': ht of all last layer units i.e., h^L_t; t=0,1,2,...,T. Its size is [seqLength,batchSize,hiddenDim*nDir]; 'nDir' is always 1 for the decoder
    #2. 'h': hT of all layers stacked together i.e., ht of the last units of all layers. Its size is [L*nDir,batchSize,hiddenDim]; 'nDir' is always 1 for the decoder
    #3. 'c': cT of all layers stacked together i.e., ct of the last units of all layers. Its size is [L*nDir,batchSize,hiddenDim]; 'nDir' is always 1 for the decoder
    op,(h,c)=self.lstm(embedding,(h,c))   #initial 'h' and 'c' are not assumed to be zeros since they are passed explicity to nn.LSTM
    #As the decoder predicts the target sentence one word (current time-step) at
    #a time, 'op' is stripped of the sequence length dimension before it is fed
    #to the dense layer.
    yHat=self.linear(op.squeeze(0))        #size is [batchSize,opDim]
    return yHat,h,c

### The sequence-to-sequence architecture
The architecture's

In [26]:
class Seq2seq(nn.Module):
  #Initializes the sequence-to-sequence architecture class
  def __init__(self,encoder,decoder,device) -> None:
    super(Seq2seq,self).__init__()
    self.encoder=encoder
    self.decoder=decoder
    self.device=device
    assert(encoder.hiddenDim==decoder.hiddenDim),'The hidden dimensions of the encoder and the decoder must be equal'
    assert(encoder.L==decoder.L),'The number of hidden layers of the encoder and the decoder must be equal'

  #Forward propagates a batch of sentences in the source language, 'source',
  #through the network
  def forward(self,source,target,ratio=0):
    #I/P sizes of the network are:
    #1. 'source': A batch of sentences in the source language. Its size is [souSeqLength,batchSize].
    #2. 'target': The batch of true sentences corresponding to 'source' in the target language. Its size is [tarSeqLength,batchSize]
    targetLn,batchSize=target.shape[0],target.shape[1]
    targetVocabSize=self.decoder.opDim                #the target language's vocabulary size
    #Pass 'source' through the encoder
    h,c=self.encoder(source)
    #Pass the context vector and the teacher-forced target language word for the
    #immediate previous time-step through the decoder. Note that the decoder
    #predicts words one time step at a time, requiring the use of a loop
    ops=torch.zeros(targetLn,batchSize,targetVocabSize).to(self.device)#stores decoder outputs
    ip=target[0,:]                                    #decoder I/P for the first time-step for all sequences in the batch is <sos>
    for t in range(1,targetLn):
      #O/P sizes of the decoder have been explained previously
      op,h,c=self.decoder(ip,h,c)
      ops[t]=op                                       #store 'op' of size [batchSize,opDim] in 'ops' along the 'targetLn' dimension
      tfFlag=random.random()<ratio                    #flag for teacher-forcing learning
      if tfFlag:ip=target[t]                          #teacher-forced I/P is true immediate previous word if flag is set, and predicted immediate previous word otherwise
      else:ip=op.argmax(1)                            #token with the highest probability score is the prediction for the current time-step. It also becomes teacher-forced I/P for the next time-step is flag is not set
    return ops

## Model training and evaluation
The functions that train, validate, and evaluate the LSTM model are defined below.

In [27]:
#This function trains and validates the model
def train(enVocabulary,deVocabulary,trDataLoader,valDataLoader,params):
  #Define the model and initialize its weights
  encoder=Encoder(params['ipDim'],params['encEmbDim'],params['hiddenDim'],params['numLayers'],params['encDORate'])
  decoder=Decoder(params['opDim'],params['decEmbDim'],params['hiddenDim'],params['numLayers'],params['decDORate'])
  model=Seq2seq(encoder,decoder,params['device']).to(params['device'])
  model.apply(initWeights)
  #Define the cost-function and the optimizer
  optimizer=optim.Adam(model.parameters(),lr=params['alpha'])
  costFun=nn.CrossEntropyLoss(ignore_index=params['enIdxPad']).to(params['device'])
  #Start training the model
  metrics=collections.defaultdict(list)
  bestValLoss=float('inf')
  model.to(params['device'])                                  #move the model to the GPU
  model.train()                                               #configure the model for training
  for epoch in range(params['epochs']):
    model.train()                                             #configure the model for training
    trLosses=[]
    #Loop over each training batch
    for batch in tqdm.tqdm(trDataLoader,desc='Model training in progress...'):
      source=batch['deIndices'].to(params['device'])
      target=batch['enIndices'].to(params['device'])
      #Forward propagation
      ops=model(source,target,params['teacherForcingRatio'])  #size is [tarSeqLength,batchSize,opDim]
      ops=ops[1:].view(-1,params['opDim'])                    #strip off <sos> from predicted target before computing loss. New shape is [tarSeqLength-1,batchSize,opDim]
      target=target[1:].view(-1)                              #change size of 'target' from [tarSeqLength,batchSize] to [(tarSeqLength-1)*batchSize]
      #Backpropagation of loss
      loss=costFun(ops,target)
      optimizer.zero_grad()                                   #initialize all gradients to zero
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(),params['clip'])
      optimizer.step()
      #Update the losses
      trLosses.append(loss.item())
    metrics['train_losses'].append(np.mean(trLosses))
    #Validate the model after each epoch
    model.eval()                                              #configure the model for evaluation
    valLosses=[]
    with torch.no_grad():                                     #backpropagation not required
      for batch in tqdm.tqdm(valDataLoader,desc='Model validation in progress...'):
        source=batch['deIndices'].to(params['device'])
        target=batch['enIndices'].to(params['device'])
        #Forward propagation
        ops=model(source,target)                              #teacher-forcing is 0 for validation and size is [tarSeqLength,batchSize,opDim]
        ops=ops[1:].view(-1,params['opDim'])
        target=target[1:].view(-1)
        #Validation loss
        loss=costFun(ops,target)
        valLosses.append(loss.item())
    metrics['valid_losses'].append(np.mean(valLosses))
    if np.mean(valLosses)<bestValLoss:
      bestValLoss=np.mean(valLosses)
      torch.save(model.state_dict(),'seq2seq.pt')
    print(f'Epoch: {epoch}')
    print(f'Training loss: {np.mean(trLosses):.3f}')
    print(f'Validation loss: {np.mean(valLosses):.3f}\n\n')
  return model,costFun

#This function initializes the network's weights from a uniform distribution
def initWeights(model):
  for layer,weights in model.named_parameters():
    nn.init.uniform_(weights.data,-8e-2,8e-2)

#This function evaluates the model
def evaluate(model,enVocabulary,deVocabulary,tsDataLoader,costFun,params):
  model.eval()                                                #configure the model for evaluation
  tsLosses=[]
  with torch.no_grad():                                       #backpropagation not required
    for batch in tqdm.tqdm(tsDataLoader,desc='Model evaluation in progress...'):
      source=batch['deIndices'].to(params['device'])
      target=batch['enIndices'].to(params['device'])
      #Forward propagation
      ops=model(source,target)                              #teacher-forcing is 0 for validation and size is [tarSeqLength,batchSize,opDim]
      ops=ops[1:].view(-1,params['opDim'])
      target=target[1:].view(-1)
      #Evaluation loss
      loss=costFun(ops,target)
      tsLosses.append(loss.item())
  return np.mean(tsLosses)

#This function translates sentences in one langugage to another andcalculates
#the BLEU loss
def translate(model,dataset,sourceTokenizer,targetTokenizer,sourceVocab,
              targetVocab,sos,eos,lower,device,maxOPLn=25):
  model.eval()                                                #configure the model for evaluation
  translations=[]
  #Start translating the sentences in the dataset
  with torch.no_grad():                                       #backpropagation not required
    for sentence in tqdm.tqdm(dataset):
      sentence=sentence['de']
      if isinstance(sentence,str):
        tokens=[token.text for token in targetTokenizer.tokenizer(sentence)]
      else:tokens=[]
      if lower:tokens=[token.lower() for token in tokens]
      tokens=[sos]+tokens+[eos]
      indices=targetVocab.lookup_indices(tokens)
      indices=torch.LongTensor(indices).unsqueeze(-1).to(device)
      h,c=model.encoder(indices)
      ips=sourceVocab.lookup_indices([sos])
      for _ in range(maxOPLn):
        op,h,c=model.decoder(torch.LongTensor([ips[-1]]).to(device),h,c)
        yHat=op.argmax(-1).item()
        ips.append(yHat)
        if yHat==sourceVocab[eos]:break
      tokens=sourceVocab.lookup_tokens(ips)
    translations.append(tokens)
  #Form the translations from tokens
  translationsHat=[''.join(translation[1:-1]) for translation in translations]
  translationsTrue=[[sentence['en']] for sentence in dataset]
  return translationsHat,translationsTrue

#This function returns the tokenizer function to claculate the BLEU loss
def getTokenizer(tokenizer,lower):
  def tokenizerFun(sentence):
    tokens=[token.text for token in tokenizer.tokenizer(sentence)]
    if lower():tokens=[token.lower() for token in tokens]
    return tokens
  return tokenizerFun

## Main function

In [28]:
if __name__=='__main__':
  device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  params={'device':device,
          'epochs':10,
          'alpha':1e-3,
          'batchSize':128,
          'clip':1,
          'teacherForcingRatio':5e-1,#percentage time for which immediate previous groud-truth is used by decoder instead of immediate previous prediction
          'ipDim':len(deVocabulary),
          'opDim':len(enVocabulary),
          'encEmbDim':256,
          'decEmbDim':256,
          'hiddenDim':512,
          'numLayers':2,
          'encDORate':5e-1,
          'decDORate':5e-1,
          'enIdxUnk':enIdxUnk,
          'enIdxPad':enIdxPad,
          'deIdxUnk':deIdxUnk,
          'deIdxPad':deIdxPad}
#Data iterators
trIterator=dataIterator('train',trSet,params)
valIterator=dataIterator('validation',valSet,params)
tsIterator=dataIterator('test',tsSet,params)
#Train the sequence-to-sequence model for language translation
model,costFun=train(enVocabulary,deVocabulary,trIterator,valIterator,params)
print('Model training is complete, and the model has {} learnable parameters.\n\n'.format(sum(weight.numel() for weight in model.parameters() if weight.requires_grad==True)))
#Evaluate the sequence-to-sequence model
model.load_state_dict(torch.load('seq2seq.pt'))
tsLoss=evaluate(model,enVocabulary,deVocabulary,tsIterator,costFun,params)
print(f'Testing loss: {tsLoss:.3f}\n\n')
#Evaluate the sequence-to-sequence model's langugage translation performance
#translations=[translate(model,sentence['de'],enTokenizer,deTokenizer,
#                        enVocabulary,deVocabulary,sosToken,eosToken,lower,
#                        params['device']) for sentence in tqdm.tqdm(tsSet)]
bleu=evaluate.load('bleu')
transHat,trans=translate(model,tsSet,enTokenizer,deTokenizer,enVocabulary,
                       deVocabulary,sosToken,eosToken,lower,params['device'])
tokenizerFun=getTokenizer(enTokenizer,lower)
evaluation=bleu.compute(predictions=transHat,references=trans,tokenizer=tokenizerFun)

Model training in progress...:   0%|          | 1/227 [00:00<00:45,  4.95it/s]

Batch sizes for English and German are torch.Size([30, 128]) and torch.Size([28, 128]) respectively.
Batch sizes for English and German are torch.Size([29, 128]) and torch.Size([32, 128]) respectively.


Model training in progress...:   1%|▏         | 3/227 [00:00<00:38,  5.84it/s]

Batch sizes for English and German are torch.Size([30, 128]) and torch.Size([27, 128]) respectively.
Batch sizes for English and German are torch.Size([27, 128]) and torch.Size([33, 128]) respectively.


Model training in progress...:   2%|▏         | 5/227 [00:00<00:37,  5.88it/s]

...

Model evaluation in progress...:  50%|█████     | 4/8 [00:00<00:00, 12.24it/s]

Batch sizes for English and German are torch.Size([35, 128]) and torch.Size([29, 128]) respectively.
Batch sizes for English and German are torch.Size([31, 128]) and torch.Size([29, 128]) respectively.
Batch sizes for English and German are torch.Size([31, 128]) and torch.Size([28, 128]) respectively.


Model evaluation in progress...: 100%|██████████| 8/8 [00:00<00:00, 13.30it/s]

Batch sizes for English and German are torch.Size([31, 128]) and torch.Size([29, 128]) respectively.
Batch sizes for English and German are torch.Size([24, 104]) and torch.Size([28, 104]) respectively.
Testing loss: 3.972




AttributeError: 'function' object has no attribute 'load'